In [1]:
import os
import folium
import openpyxl
import pandas as pd
import geopandas as gpd

In [13]:
autos = pd.read_csv('Ausgabedateien\autos_aufbereitet.csv') # File aus Autos_Datenaufbereitung.ipynb

OSError: [Errno 22] Invalid argument: 'Ausgabedateien\x07utos_aufbereitet.csv'

In [ ]:
autos.head(2)

,Unnamed: 0,web-scraper-order,web-scraper-start-url,Preis,plz,jahr,ps-stand,Farbe,PLZ4,%_IN_GDE,KTKZ,GDENR,GDENAMK,Hist.-Nummer,Kanton,Bezirks-nummer,Bezirksname,BFS Gde-nummer,Gemeindename,Datum der Aufnahme
0,0,1669622809-189,https://www.autolina.ch/elektro-auto,49'650,8640,NaN,231.0,silber,8640.0,3.91,SZ,1322.0,Freienbach,11961.0,SZ,503,Höfe,1322,Freienbach,1848-09-12
1,1,1669622809-190,https://www.autolina.ch/elektro-auto,42'500,6464,2022.0,204.0,weiss,6464.0,0.36,UR,1205.0,Bürglen (UR),11506.0,UR,400,Kt. Uri,1205,Bürglen (UR),1848-09-12


In [ ]:
autos = autos.astype({"Bezirks-nummer":int})
autos.head(2)

,Unnamed: 0,web-scraper-order,web-scraper-start-url,Preis,plz,jahr,ps-stand,Farbe,PLZ4,%_IN_GDE,KTKZ,GDENR,GDENAMK,Hist.-Nummer,Kanton,Bezirks-nummer,Bezirksname,BFS Gde-nummer,Gemeindename,Datum der Aufnahme
0,0,1669622809-189,https://www.autolina.ch/elektro-auto,49'650,8640,NaN,231.0,silber,8640.0,3.91,SZ,1322.0,Freienbach,11961.0,SZ,503,Höfe,1322,Freienbach,1848-09-12
1,1,1669622809-190,https://www.autolina.ch/elektro-auto,42'500,6464,2022.0,204.0,weiss,6464.0,0.36,UR,1205.0,Bürglen (UR),11506.0,UR,400,Kt. Uri,1205,Bürglen (UR),1848-09-12


In [ ]:
# Polygonmap als .json-File
polys = gpd.read_file("bezirke.csv")

# Structure and type
print("nrows, ncols", polys.shape)
print("-------------------------------------------------------")
print("Type:", type(polys))

# Object 'polys' is a GeoDataFrame
polys.head()

nrows, ncols (676, 10)
-------------------------------------------------------
Type: <class 'geopandas.geodataframe.GeoDataFrame'>


,Geo Point,Geo Shape,Year,Official code Kanton,Official Name Kanton,Official code Bezirk,Official Name Bezirk,Iso 3166-3 Area Code,Type,geometry
0,"46.26224041296291,6.872836741406499","{""coordinates"": [[[6.821069658830559, 46.42715...",2022,23,Valais,2308,Monthey,CHE,Bezirk,None
1,"46.801859618327995,6.90940789390998","{""coordinates"": [[[[6.773010442473133, 46.7513...",2022,22,Vaud,2222,Broye-Vully,CHE,Bezirk,None
2,"47.711620153190225,8.599978677886451","{""coordinates"": [[[[8.595727904980112, 47.6056...",2022,14,Schaffhausen,1403,Schaffhausen,CHE,Bezirk,None
3,"46.10489655647573,7.844097748265566","{""coordinates"": [[[[7.881083947624592, 46.3184...",2022,23,Valais,2313,Visp,CHE,Bezirk,None
4,"47.140599819190484,8.902637247572136","{""coordinates"": [[[8.886088147494995, 47.07074...",2022,05,Schwyz,0505,March,CHE,Bezirk,None


In [ ]:
Angebote_pro_gemeinde = pd.DataFrame(autos['Bezirks-nummer'].value_counts().rename_axis('Bezirk').reset_index(name='Anzahl'))
Angebote_pro_gemeinde

,Bezirk,Anzahl
0,110,120
1,2014,108
2,1727,81
3,103,76
4,312,76
...,...,...
109,1110,1
110,1844,1
111,2304,1
112,501,1


In [ ]:
# Initialisierung der Map
m = folium.Map(location=[47.44, 8.65], zoom_start=10)

# Map settings
folium.Choropleth(
    geo_data=polys,
    name='polys',
    fill_color='greenyellow'
).add_to(m)

folium.LayerControl().add_to(m)

# Plot map
#m

c:\Users\jkunz\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\array.py:938: RuntimeWarning: All-NaN slice encountered
  np.nanmin(b[:, 0]),  # minx
c:\Users\jkunz\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\array.py:939: RuntimeWarning: All-NaN slice encountered
  np.nanmin(b[:, 1]),  # miny
c:\Users\jkunz\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\array.py:940: RuntimeWarning: All-NaN slice encountered
  np.nanmax(b[:, 2]),  # maxx
c:\Users\jkunz\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\array.py:941: RuntimeWarning: All-NaN slice encountered
  np.nanmax(b[:, 3]),  # maxy


ValueError: Cannot transform naive geometries.  Please set a crs on the object first.

In [ ]:
# Variable to plot
var = 'Anzahl'

# Function for hiding the default legend
def folium_del_legend(choropleth: folium.Choropleth):
    del_list = []
    for child in choropleth._children:
        if child.startswith('color_map'):
            del_list.append(child)
            for del_item in del_list:
                choropleth._children.pop(del_item)
                return choropleth

# Bins for the color range
bins = list(Angebote_pro_gemeinde[var].quantile([0.00, 0.3, 0.5, 0.8, 1.00]))

# Initialize map
m = folium.Map(location=[47.44, 8.65], zoom_start=10)

# Map-Settings (key_on contains the key to match the attribute data)
folium.Choropleth(
        geo_data=polys,
        name='choropleth',
        data=Angebote_pro_gemeinde,
        columns=['BFS', var],
        key_on='feature.properties.BFS',
        fill_color='RdGy',
        fill_opacity=0.7,
        line_opacity=0.5,
        legend_name=var,
        bins=bins,
        reset=True
).add_to(m)

# Layer controls
folium.LayerControl().add_to(m)

# Plotting the map
m